# Start

In [1]:
import dask.dataframe as dd
from dask.distributed import Client
from dask.diagnostics import ProgressBar

/anaconda3/envs/overscripted/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda3/envs/overscripted/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


All sub samples and new samples with new columns/data will be saved under the "DIR" directory to keep things organized. 
As such, the function "save_parquet" and "read_parquet" adds this directory to every parquet name, and I'm using this functions instead of dd.read_parquet/dd.to_parquet direct to ensure the same read and write settings across the notebook. 

In [2]:
#Initializing client / distributed
# client = Client()
# client

#Create folder to save/read new data
DIR = 'sample_0_prep/'
import os
if not os.path.exists(DIR):
    os.makedirs(DIR)

If no "recalculate_partition" is passed on, it will not recalculate the partitions. It is not mandatory, but good if you are significantly reducing the size of the data. 

In [3]:
#Save a DF to a parquet
def save_parquet(df, name, recalculate_partition=False):
    with ProgressBar():
        #DF.REPARTITION copyed from: https://stackoverflow.com/questions/44657631/strategy-for-partitioning-dask-dataframes-efficiently
        if recalculate_partition:
            n = 1+df.memory_usage(deep=True).sum().compute() // (1000 * 1000 * 100)
            print("Npartition: ", n)
            df.repartition(npartitions= n).to_parquet(DIR + name, engine="pyarrow")
        else:
            df.to_parquet(DIR + name, engine="pyarrow")
        
        
def read_parquet(name):
    return dd.read_parquet(DIR + name, engine='pyarrow')

# Data
Using 10% sample and self produced samples
 - 10% sample has 11292867 rows
 - Filtered by value_len > df.mean() has 499805 rows

In [4]:
#Original sample
df = dd.read_parquet('sample_0.parquet', 
                     engine='pyarrow', 
                     columns=['value_1000', 'value', 'value_len', 'symbol', 'script_url', 'location'])

# df.astype({'value_1000': str, 'value': str,'value_len': int,'symbol': int,'script_url': str})
df.columns

Index(['value_1000', 'value', 'value_len', 'symbol', 'script_url', 'location'], dtype='object')

## DF overview
Some overview about the sample: 
- Mean: 1356.97,
- Min: 0,
- Max: 4496861
- Std: 26310.62

In [5]:
with ProgressBar():
    df_mean = df['value_len'].mean()
    df_min = df['value_len'].min()
    df_max = df['value_len'].max()
    df_std = df['value_len'].std()
    df_len = df['value_len'].count()
    (df_mean, df_min, df_max, df_std, df_len) = dd.compute(df_mean, df_min, df_max, df_std, df_len);
    print(df_mean, df_min, df_max, df_std, df_len)

[########################################] | 100% Completed | 48.2s
1356.9776628910975 0 4496861 26310.62140481331 11292867


### FILTER: value_len > df_mean
1356 is the value_len mean

To filter the data into something that is more interesting to this task I decided to only work with values that are at above the mean.

All values above the mean count up to 499805 rows. That is just 4,42% of the whole sample, and a lot easier to work on. 

In [6]:
#Save
save_parquet(df= df[df['value_len'] > df_mean], name='above_mean.parquet', recalculate_partition=True)

[########################################] | 100% Completed | 58.0s
Npartition:  244
[########################################] | 100% Completed |  1min 30.9s


In [7]:
#Read
df = read_parquet('above_mean.parquet')
df.columns

Index(['value_1000', 'value', 'value_len', 'symbol', 'script_url', 'location'], dtype='object')

# Add Column: Domains
The following code is copyed from this same project: ~/analyses/hello_world.ipynb

It uses the data saved from the last section
This section is dedicated to extract the domain of the columns "location" and "script_url" and add it as new columns "location_domain" and "script_domain"

In [8]:
import tldextract

def extract_domain(url):
    """Use tldextract to return the base domain from a url"""
    try:
        extracted = tldextract.extract(url)
        return '{}.{}'.format(extracted.domain, extracted.suffix)
    except Exception as e:
        return 'ERROR'

In [9]:
#To guarantee the usage of the correct parquet created above in case we start from this section
df = read_parquet('above_mean.parquet')

df.astype({'value_1000': str, 'value': str,'value_len': int,'symbol': int,'script_url': str, 'location': str})
df['location_domain'] = df.location.apply(extract_domain, meta='O')
df['script_domain'] = df.script_url.apply(extract_domain, meta='O')

In [10]:
#save
save_parquet(df=df, name='above_mean_domain.parquet')

[########################################] | 100% Completed |  1min 17.3s


In [11]:
#read
df = read_parquet('above_mean_domain.parquet')
df[['location_domain',  'location', 'script_domain', 'script_url']].head()

,location_domain,location,script_domain,script_url
0,canada.ca,https://www.canada.ca/en/services.html,adobedtm.com,https://assets.adobedtm.com/caacec67651710193d...
1,tmall.com,https://maniform.world.tmall.com/category-1282...,alicdn.com,https://g.alicdn.com/alilog/mlog/aplus_v2.js
2,tmall.com,https://maniform.world.tmall.com/category-1282...,alicdn.com,https://g.alicdn.com/alilog/mlog/aplus_v2.js
3,coches.net,https://www.coches.net/fiat/segunda-mano/,coches.net,https://www.coches.net/scripts/common.min.js?2...
4,coches.net,https://www.coches.net/fiat/segunda-mano/,coches.net,https://www.coches.net/scripts/common.min.js?2...


# Add Column:  is_json

After manual initial analysis I have think that the huge values are json structured, to validate that I included an new column that is a boolean value with the validation of json

After simple validation of value is a json or not, boolean value will be saved on a new column named "is_json"


In [12]:
import json
import pandas as pd

def is_json(myjson):
    try:
        json.loads(myjson)
        return True

    except ValueError as e:
        return False

In [13]:
#To guarantee the usage of the correct parquet created above in case we start from this section
df = read_parquet('above_mean_domain.parquet')
df['is_json'] = df['value'].apply(is_json, meta=False)

In [14]:
#save
save_parquet(df=df, name='above_mean_domain_json.parquet')

[########################################] | 100% Completed |  2min 25.1s


In [15]:
#read
df = read_parquet('above_mean_domain_json.parquet')
df[['value_1000', 'is_json']].head()

,value_1000,is_json
0,"{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...",True
1,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",True
2,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",True
3,usunico=17/12/2017:0-00155123:830; SessionASM=...,False
4,usunico=17/12/2017:0-00155123:830; SessionASM=...,False


# Add Column:  value_md5
Include new columns called "value_md5" that is the md5 of value column

In [16]:
import hashlib

def md5(value):
    return hashlib.md5(value.encode('utf-8')).hexdigest()

In [17]:
#To guarantee the usage of the correct parquet created above in case we start from this section
df = read_parquet('above_mean_domain_json.parquet') 

df['value_md5'] = df['value'].apply(md5, meta=' ')

In [18]:
#save
save_parquet(df=df, name='above_mean_domain_json_md5.parquet')

[########################################] | 100% Completed |  1min 26.8s


In [19]:
#read
df = read_parquet('above_mean_domain_json_md5.parquet')
df[['value_1000', 'value_md5']].head()

,value_1000,value_md5
0,"{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...",cff77029e3ae45dd439a62987b1d8340
1,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",9ac0a0a0afb677c8fd985a7c2f4ddbc5
2,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",9ac0a0a0afb677c8fd985a7c2f4ddbc5
3,usunico=17/12/2017:0-00155123:830; SessionASM=...,db64465b639e01993d9212390f057628
4,usunico=17/12/2017:0-00155123:830; SessionASM=...,db64465b639e01993d9212390f057628


# Saving other possible usefull samples to future analyses

### Filter to parquet containing only JSON 

In [21]:
df = read_parquet('above_mean_domain_json_md5.parquet')
save_parquet(df=df[df['is_json'] == True], name='JSONs_only.parquet', recalculate_partition=True)

[########################################] | 100% Completed | 27.4s
Npartition:  233
[########################################] | 100% Completed |  1min  3.4s


In [22]:
#read all_json_above_mean
df = read_parquet('JSONs_only.parquet')
df[['value_1000', 'is_json']].head()

,value_1000,is_json
0,"{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...",True
1,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",True
2,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",True
3,"{""dueljs_channel_comm"":""[{\""id\"":4734405521279...",True
4,"{""dueljs_channel_comm"":""[{\""id\"":4734405521279...",True


### Add json keys and schema columns

Extract the top level keys, sort them and add as a list into another column named 'json_keys'
Will be using "https://github.com/rnd0101/json_schema_inferencer" to guess the json schema and save it into another column called "json_schema"

In [23]:
from json_schema_inferencer.guess_json_schema import guess_schema

df = read_parquet('JSONs_only.parquet')

def jsonSchema(myjson):
    try:
        dct = json.loads(myjson)
        value = guess_schema(dct)
        l = list(value['properties'])
        l.sort()
        return l
    except ValueError as e:
        return list()
    
def jsonKeys(myjson):
    try:
        dct = json.loads(myjson)
        keys = list(dct.keys())
        keys.sort()
        return keys
    except ValueError as e:
        return list()

In [24]:
df['json_keys'] = df['value'].apply(jsonKeys, meta='')
df['json_schema'] = df['value'].apply(jsonSchema, meta='')
save_parquet(df=df, name='JSONs_key_schema.parquet')

[########################################] | 100% Completed |  3min 57.7s


In [25]:
#read 
df = read_parquet('JSONs_key_schema.parquet')
df[['value_1000', 'json_keys', 'json_schema']].head()

,value_1000,json_keys,json_schema
0,"{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...",[im-settings],[im-settings]
1,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",[APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c],[APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c]
2,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",[APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c],[APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c]
3,"{""dueljs_channel_comm"":""[{\""id\"":4734405521279...","[LastSearch, LastSearch_e, dueljs_channel_comm...","[LastSearch, LastSearch_e, dueljs_channel_comm..."
4,"{""dueljs_channel_comm"":""[{\""id\"":4734405521279...","[LastSearch, LastSearch_e, dueljs_channel_comm...","[LastSearch, LastSearch_e, dueljs_channel_comm..."


### All NON json above the mean

In [27]:
df = read_parquet('above_mean_domain_json_md5.parquet')
save_parquet(df=df[df['is_json'] == False], name='NON_JSONs_only.parquet', recalculate_partition=True)

[########################################] | 100% Completed | 26.7s
Npartition:  12
[########################################] | 100% Completed | 27.8s


In [28]:
#read 
df = read_parquet('NON_JSONs_only.parquet')
df.head()

,value_1000,value,value_len,symbol,script_url,location,location_domain,script_domain,is_json,value_md5
0,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,https://www.coches.net/fiat/segunda-mano/,coches.net,coches.net,False,db64465b639e01993d9212390f057628
1,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,https://www.coches.net/fiat/segunda-mano/,coches.net,coches.net,False,db64465b639e01993d9212390f057628
2,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://tags.tiqcdn.com/utag/schibsted/coches....,https://www.coches.net/fiat/segunda-mano/,coches.net,tiqcdn.com,False,db64465b639e01993d9212390f057628
3,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://tags.tiqcdn.com/utag/schibsted/coches....,https://www.coches.net/fiat/segunda-mano/,coches.net,tiqcdn.com,False,db64465b639e01993d9212390f057628
4,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://tags.tiqcdn.com/utag/schibsted/coches....,https://www.coches.net/fiat/segunda-mano/,coches.net,tiqcdn.com,False,db64465b639e01993d9212390f057628
